# Treinando modelo de regressão logística com SDK do Azure para Python

## Verificar o pacote **azure-ai-ml**

É necessário a última versão do pacote **azure-ai-ml** para rodar o código nesse notebook. Executar a célula abaixo para verificar se está na última versão.

> **Obs**:
> Se o pacote **azure-ai-ml** não estiver instalado, execute `pip install azure-ai-ml` para instalar.

In [3]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.26.3
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-monitor-opentelemetry, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Conectar workspace do Azure Machine Learning

Para conectar a workspace é preciso ter antes criado os seguintes itens: Resource Group > Workspace Azure Machine Learning > Instância de Computação e Cluster.

No script será necessário identificar os parâmetros, mas ao executar esse notebook dentro de um Studio do Azure Machine Learning, esses podem ser setados como Default.

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Verifica se consegue gerar token
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Em caso de falha, o Browser pedirá as credenciais.
    credential = InteractiveBrowserCredential()

In [ ]:
# Configurando a workspace
ml_client = MLClient.from_config(credential=credential)

## Treinando o Modelo

Para treinar um modelo, é preciso criar o script python (**vendas-sorvetes.py**) na pasta **src**. O script utilizará o arquivo **vendas_sorvetes_clima_itanhaem_2023.csv** da mesma pasta.

In [4]:
%%writefile src/vendas-sorvetes.py
# importar bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load dos dados de vendas
print("Loading Data...")
diabetes = pd.read_csv('vendas_sorvetes_clima_itanhaem_2023.csv')

# separar features e labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split dos dados entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# configurar hyperparâmetro de regularização
reg = 0.01

# treinar modelo de regressão logística
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calcular acurácia
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calcular AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))


Writing src/vendas-sorvetes.py


FileNotFoundError: [Errno 2] No such file or directory: 'src/vendas-sorvetes.py'

Run the cell below to submit the job that trains a classification model to predict diabetes. 

In [ ]:
from azure.ai.ml import command

# configurar o job
job = command(
    code="./src",
    command="python sorvetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="computeml",
    display_name="sorvetes-pythonv2-train",
    experiment_name="sorvetes-training"
)

# criar job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)